In [1]:
import numpy as np
# import pandas as pd
# import math
from scipy.stats import multivariate_normal

In [2]:
from sklearn.mixture import GMM as GaussianMixture
# from sklearn.mixture import StratifiedKFold?

In [3]:
# import sys
# sys.path.append("..")
# from clustering.clustering import kmeans_cal
from sklearn.cluster import KMeans

In [4]:
from random import randint

## Functions

## _GMM_

### LIB

In [5]:
def  find_clusters(items, features, n_cluster, n_iter):
    '''
        itemset: list indexes of images corresponding to video_img. Discard indexes where feature is None.
        usr_list: |U| users, each user is a sets of items.
        feature: gist or lab feature coressponding image in video_img.
    '''
    kmeans = KMeans(n_clusters = n_cluster, random_state=0).fit(features[items])
    label_init = kmeans.labels_

    gmm = GaussianMixture(n_components=n_cluster, covariance_type = 'full', n_iter = n_iter, random_state=0)
    gmm.means_init = np.array([features[items[label_init == i]].mean(axis=0) for i in range(n_cluster)])
    gmm.fit(features[items])
    label_pred = gmm.predict(features[items])
    clusters = [np.where(label_pred == i)[0] for i in range(n_cluster)]

    means = gmm.means_
#     covs = gmm.covariances_
    covs = gmm.covars_
    weights = gmm.weights_
    
    p_x_y = np.vstack([np.apply_along_axis(multivariate_normal.pdf, 1, features[[items]], means[k], covs[k], True) \
                                                for k in range(n_cluster)]).T#     
    p_x = np.sum(p_x_y * np.array(weights).reshape(1, -1) , axis = 1).reshape(-1, 1) 
    print 'Log Likelihood: ', np.sum(np.log(p_x))
    return np.sum(np.log(p_x)), gmm

### My GMM

In [6]:
def cal_meancov(features, items, label, n_cluster):
    mean = [np.mean(features[items[label == i]], axis = 0) for i in range(n_cluster)]
    cov = [np.cov(features[items[label == i]], rowvar = 0) for i in range(n_cluster)]
    return mean, cov

In [ ]:
# def  my_gmm(items, features, n_cluster):
#     kmeans = KMeans(n_clusters = n_cluster, random_state=0).fit(features[items])
#     label_init = kmeans.labels_
#     mean, cov = cal_meancov(features, items, label_init, n_cluster)
#     p_y = np.array([1.0/n_cluster] * n_cluster).reshape(-1, 1)

#     iter = 10
#     converge = False
#     while (iter > 0 and not converge):
#         # e step
#         p_x_y = np.vstack([np.apply_along_axis(multivariate_normal.pdf, 1, features[items], mean[k], cov[k], True) \
#                                                 for k in range(n_cluster)]) # K * N
#         p_y_x = p_x_y * p_y # K * N
# #         p_y_x_bug = np.array([np.array([p_x_y[k][i] * p_y[k] for i in items]) for k in range(n_cluster)]).reshape(n_cluster, len(items))
#         p_x = np.sum(p_y_x, axis = 0).reshape(-1, 1) # N * 1 
# #         p_y_x_bug = np.array([p_y_x_bug[k] / p_x[k] for k in range(n_cluster)]).reshape(n_cluster, len(items))
#         p_y_x = p_y_x / p_x.reshape(-1,) # K * N
        
# #         print np.mean(p_y_x_bug - p_y_x, axis = 1)

#         print 'Log prob: ', np.sum(np.log(p_x))
        
#         # m step       
#         sum_weight = np.sum(p_y_x, axis = 1).reshape(-1, 1)  # K x 1
#         mean = np.dot(p_y_x, features[items]) * (1.0/sum_weight.reshape(-1,1)) # K x F
# #         mean_bug = np.array([np.sum([p_y_x[k][i] * features[i] for i in items], axis = 0) /sum_weight[k] for k in range(n_cluster)])
# #         print np.mean(mean_bug - mean, axis = 1)
#         x_submean = np.array([features[items] - mean[k] for k in range(n_cluster)])
#         cov = [np.dot(np.multiply(x_submean[k].T, p_y_x[k]), x_submean[k]) / sum_weight[k] for k in range(n_cluster)] # K x F x F
# #         cov = [np.sum([np.multiply(x_submean[k][i].reshape(-1, 1), p_y_x[k][i]) * x_submean[k][i].reshape(1,-1) \
# #                            for i in items], axis = 0) / sum_weight[k] for k in range(n_cluster)]
#         p_y = sum_weight / len(items)  # K x 1
        
#         iter -= 1
#     print 'Log prob: ', np.sum(np.log(p_x))
#     return np.sum(np.log(p_x)), (p_y, mean, cov)

In [13]:
def  my_gmm(items, features, n_cluster):
    kmeans = KMeans(n_clusters = n_cluster, random_state=0).fit(features[items])
    label_init = kmeans.labels_
    mean, cov = cal_meancov(features, items, label_init, n_cluster)
    p_y = np.array([1.0/n_cluster] * n_cluster).reshape(-1, 1)

    p_x_y = np.zeros((n_cluster, len(items))) # K * N
    p_y_x = np.zeros((n_cluster, len(items))) # K * N
    
    iter = 10
    converge = False
    while (iter > 0 and not converge):
        # e step
        for k in range(n_cluster):
            p_x_y[k, :] = np.apply_along_axis(multivariate_normal.pdf, 1, features[items], mean[k], cov[k], True)
        p_y_x = p_x_y * p_y # K * N
        p_x = np.sum(p_y_x, axis = 0).reshape(-1, 1) # N * 1 
        p_y_x = p_y_x / p_x.reshape(-1,) # K * N
        log_likelihood = np.sum(np.log(p_x))

        print 'Log Likelihood: ', log_likelihood
        
        # m step       
        sum_weight = np.sum(p_y_x, axis = 1).reshape(-1, 1)  # K x 1
        mean = np.dot(p_y_x, features[items]) * (1.0/sum_weight.reshape(-1,1)) # K x F
        x_submean = np.array([features[items] - mean[k] for k in range(n_cluster)])
        cov = [np.matmul(np.multiply(x_submean[k].T, p_y_x[k]), x_submean[k]) / sum_weight[k] for k in range(n_cluster)] # K x F x F

        p_y = sum_weight / len(items)  # K x 1
        
        iter -= 1
    print 'Log likelihood: ', np.sum(np.log(p_x))
    return np.sum(np.log(p_x)), (p_y, mean, cov)

## Debug

In [14]:
log_prob1, _ = my_gmm(np.array(range(N)), data, n_cluster = K)

Log Likelihood:  -435.52859828
Log Likelihood:  -1232.63590897
Log Likelihood:  -295.503637766
Log Likelihood:  9.32699658072
Log Likelihood:  -303.132814235
Log Likelihood:  -595.121151956
Log Likelihood:  -266.786131773
Log Likelihood:  -273.771526497
Log Likelihood:  -122.703139938
Log Likelihood:  -1179.72154832
Log likelihood:  -1179.72154832


In [11]:
for n_iter in range(1, 10):
    log_prob0, _ = find_clusters(np.array(range(N)), data, n_cluster = K, n_iter = n_iter)

Log Likelihood:  -603.337180827
Log Likelihood:  754.141079928
Log Likelihood:  829.634546552
Log Likelihood:  829.636477176
Log Likelihood:  829.636477176
Log Likelihood:  829.636477176
Log Likelihood:  829.636477176
Log Likelihood:  829.636477176
Log Likelihood:  829.636477176


## Test

In [8]:
''' GLOBAL VARIABLES '''
log_prob0s = []
log_prob1s = []
TEST_CASE = 10
N, K = None, None
data = []

In [9]:
def test(N, K, data):
    print N, K
    
    print 'LIB'
    log_prob0 = None
    for n_iter in range(1, 10):
        log_prob0, _ = find_clusters(np.array(range(N)), data, n_cluster = K, n_iter = n_iter)

    print '\nMy GMM'
    log_prob1, _ = my_gmm(np.array(range(N)), data, n_cluster = K)
    return log_prob0, log_prob1

In [10]:
for i in range(TEST_CASE):
    print '\n------------TEST %d----------------' % i
    K = randint(3, 11)
    N = randint(1, 10) * (20 - K)
    
    data = []
    locs = (np.random.randint(10, size=K) + 1) * 100
    szs = (np.random.randint(15, size=K) + 1) * 10
    f_sz = 17
    print locs
    for i in range(K):
        data = data + list(np.random.normal(loc=locs[i], scale=1.0, size=(szs[i], f_sz)))
    data = np.array(data)
    print data.shape
    
    log_prob0, log_prob1 = test(N, K, data)
    log_prob0s.append(log_prob0)
    log_prob1s.append(log_prob1)
print '\n----------------------------------'
print 'AVG %d TEST %.4f %.4f' % (TEST_CASE, np.mean(log_prob0s),  np.mean(log_prob1s))


------------TEST 0----------------
[400 700 600 800 800 100]
(370L, 17L)
14 6
LIB
Log Likelihood:  142.657084357
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055
Log Likelihood:  468.635009055

My GMM
Log Likelihood:  -68.7135423889
Log Likelihood:  -123.290731792
Log Likelihood:  -11.2571407302
Log Likelihood:  -36.9113616795
Log Likelihood:  -128.755399366
Log Likelihood:  -17.9806584705
Log Likelihood:  -29.9997849018
Log Likelihood:  -99.0722507693
Log Likelihood:  -116.98011463
Log Likelihood:  -87.7555559313
Log likelihood:  -87.7555559313

------------TEST 1----------------
[100 500 800 500 200 200 400 300 400 600]
(700L, 17L)
50 10
LIB
Log Likelihood:  -404.407866433
Log Likelihood:  983.478820683
Log Likelihood:  1045.87800554
Log Likelihood:  1058.10379959
Log Likelihood:  1058.10379959
Log Likelihood:  1058.1